In [1]:
from vector_database import get_reranked_documents
from models import load_models
import pandas as pd 
import torch
torch.set_default_device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
test_file_path:str = 'data/evaluation/synthetic_queries.csv'

In [3]:
embedding_model, reranker_model = load_models()

In [4]:
test_data = pd.read_csv(test_file_path, index_col=0)

In [5]:
test_data = test_data[~test_data['query'].str.contains('Relevanz', na=False)]

In [6]:
retrieved_document_data = {'document_in_top_5': [], 'chapter_in_top_5': [],'document_in_top_3': [], 'chapter_in_top_3': [],'document_in_top_1': [], 'chapter_in_top_1': []}
retrieval_results = []
for _,query, document_title, chapter_name in test_data.itertuples():
    results = get_reranked_documents(query, embedding_model, reranker_model)
    results['title'] = results['title'].str.lower()
    retrieval_results.append(results)
    retrieved_document_data['document_in_top_5'].append((results['title'].iloc[:5]==document_title).any())
    retrieved_document_data['chapter_in_top_5'].append((results['section'].iloc[:5]==chapter_name).any())
    retrieved_document_data['document_in_top_3'].append((results['title'].iloc[:3]==document_title).any())
    retrieved_document_data['chapter_in_top_3'].append((results['section'].iloc[:3]==chapter_name).any())
    retrieved_document_data['document_in_top_1'].append((results['title'].iloc[:1]==document_title).any())
    retrieved_document_data['chapter_in_top_1'].append((results['section'].iloc[:1]==chapter_name).any())

In [7]:
scores = {}
for k, v in retrieved_document_data.items():
    print(70*'#')
    correct_percentage = sum(v)/len(v)
    print(f'{k} Percentage: {correct_percentage:.4f}')
    scores[k] = correct_percentage
    print(70*'#' + '\n')

######################################################################
document_in_top_5 Percentage: 0.9787
######################################################################

######################################################################
chapter_in_top_5 Percentage: 0.2512
######################################################################

######################################################################
document_in_top_3 Percentage: 0.9630
######################################################################

######################################################################
chapter_in_top_3 Percentage: 0.2102
######################################################################

######################################################################
document_in_top_1 Percentage: 0.8898
######################################################################

######################################################################
chapter_in_top_1 Percentage: 0.

In [8]:
test_data.loc[~pd.Series(retrieved_document_data['document_in_top_5'])]

,query,document_title,chapter_name
38,Optimierung der psychosozialen Versorgung auf ...,s3 analgesie sedierung delirmanagement in der ...,"A. Analgesie, Sedierung und Delirmanagement be..."
132,Intensivmedizinische Versorgung von Patienten ...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 3) Brandve...
142,Intensivmedizinische Versorgung von Kindern mi...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 3a) Das br...
143,Risiken und Komplikationen von Verbrennungen b...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 3a) Das br...
150,Behandlungsschwerpunkte bei Patienten mit mehr...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...
151,Notfallmanagement bei Patienten mit multiplen ...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...
155,Diagnostik und Management von Schock bei polyt...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...
156,Intensivmedizinische Versorgung von Patienten ...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...
157,Komplikationen bei der Behandlung von Patiente...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...
158,Langzeitfolgen bei Patienten mit multiplen Tra...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...


In [9]:
retrieval_results[1216]

,document_filename,title,section,heading,text,table_flag,page_number,cosine_similarity,reranking_score
0,001-012l_S3_Analgesie-Sedierung-Delirmanagemen...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...,4) Polytraumatisierte Patient:innen,Für den Weaningpozess wurde für Traumapatient:...,False,107,0.552247,0.607197
1,001-012l_S3_Analgesie-Sedierung-Delirmanagemen...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 5) Patient...,5) Patient:innen mit erhöhtem intrakraniellen ...,"Neben einer adäquaten Analgesie, die bei schwe...",False,111,0.592554,0.558809
2,001-012l_S3_Analgesie-Sedierung-Delirmanagemen...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 5) Patient...,5) Patient:innen mit erhöhtem intrakraniellen ...,Das schwere SHT stellt eines der wichtigsten K...,False,110,0.539462,0.553747
3,001-012l_S3_Analgesie-Sedierung-Delirmanagemen...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 4) Polytra...,4) Polytraumatisierte Patient:innen,Eine effektive Sedierung polytraumatisierten P...,False,106,0.599746,0.509825
4,001-012l_S3_Analgesie-Sedierung-Delirmanagemen...,s3 analgesie sedierung delirmanagement in der ...,B. Spezielle Patient:innengruppen - 5) Patient...,Propofol,In einem monozentrischen RCT mit 100 schwer tr...,False,120,0.532770,0.499603
